## SmokeVideoCV
This notebook explores streaming videos from the [Internet Archive](https://archive.org/). 
- connect to video
- pull frame 
- analize frame for faces
- inscribe box around face

Verify that it's possible to get all the frames from an video, over the web. Answer - yes.


next step: move the processing to Streams. The processing will leverage off the [imgAna_OpenCV](imgAna_OpenCV.jupyter-py36.ipynb) notebook, which analysis images submitted to Wikipedia.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import cv2
from PIL import Image,  ImageDraw  # https://pillow.readthedocs.io/en/4.3.x/
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox, Layout
from IPython.display import display, clear_output
from PIL import Image

def convertToRGB(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#plt.imshow(convertToRGB(img_raw))

def convertToBW(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


In [ ]:
## setup for image analysis...

haar_cascade_face = cv2.CascadeClassifier('../datasets/haarcascade_frontalface_default.xml')
haar_cascade_face


In [ ]:
def line_box(ele):
    (x,y,w,h)=ele
    return (x,y, x+w,y, x+w,y+h, x,y+w, x,y)

def bts_to_img(bts):
    '''
    :param bts: results from image_to_bts
    '''
    buff = np.fromstring(bts, np.uint8)
    buff = buff.reshape(1, -1)
    img = cv2.imdecode(buff, cv2.IMREAD_COLOR)
    return img

def inscribe_rect(bin_image, detection_box):
    """Inscribe box on image
    
    This is updating the image passed in.
    
    Args:
        bin_image : binary image
        detection_box : region to put box around
    Return:
        return image - 
    """
    draw = ImageDraw.Draw(bin_image) 
    box_width = 10 
    draw.line(line_box(detection_box), fill="yellow", width=box_width)
    return bin_image

In [ ]:

# https://github.com/nasa/ISS_Camera_Geolocate.git
frame_internal = 10
minimum_faces_per_frame = 1
video_selection_url = 3

url = ['https://www.sample-videos.com/video123/mp4/720/big_buck_bunny_720p_1mb.mp4',
'https://archive.org/download/popeye_shuteye_popeye/popeye_shuteye_popeye_512kb.mp4',
'https://archive.org/download/MuhammadAliVsSonnyListon/MuhammadAliVsSonnyListon_512kb.mp4',
'https://archive.org/download/bb_snow_white/bb_snow_white_512kb.mp4',
'https://archive.org/download/TuringTe2001/TuringTe2001_256kb.mp4']

cap = cv2.VideoCapture(url[video_selection_url])

def rects_render(image, rects):
    image_copy = image.copy()
    for (x, y, w, h) in rects:
        cv2.rectangle(image_copy, (x, y), (x+w, y+h), (0, 255, 0), 2)
    return image_copy

demo_widget = widgets.Output(layout={'border': '1px solid green'})
display(demo_widget)
count = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret is False:
        break
    count += 1
    if not(count % frame_internal):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces_rects = haar_cascade_face.detectMultiScale(gray, scaleFactor = 1.2, minNeighbors = 5); 
        if len(faces_rects) >= minimum_faces_per_frame:
            with Image.fromarray(frame) as bin_image:
                for rect in faces_rects:
                    inscribe_rect(bin_image, rect)
            with demo_widget:
                display(bin_image)
                clear_output(wait=True)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## Find videos on the internet archive.....

Pulling videos from the video page, example :
- goto cartool page : cartoons : [cartoons videos](https://archive.org/details/animationandcartoons?and[]=mediatype%3A%22movies%22)
- select a cartoon - take you to new page.
- In 'DOWNLOAD OPTIONS" right click the 'MPEG..' select - 'Copy Link Location". This will copy the the link into the paste buffer this send it to the application.....